# Alguns métodos de resolução de sistemas lineares

# Decomposição LU

###### Neste método, decompomos a matriz (não singular) no produto de duas matrizes triangulares L e U.
###### Temos então:
$A*X = B \rightarrow (LU)*X = B \rightarrow L(UX) = B$
###### Chamando $UX = C$, resolvemos o problema $LC = B$, e em seguida recuperamos $X$ fazendo $UX = C$

### Temos então $L = 
\begin{bmatrix}
l_{11} & 0 & 0\\
l_{21} & l_{22} & 0\\
l_{31} & l_{32} & l_{33}\\
\end{bmatrix}$
### e
$U = \begin{bmatrix}
1 & u_{12} & u_{13}\\
0 & 1 & u_{23}\\
0 & 0 & 1\\
\end{bmatrix}$

### Para uma matriz $M$, $M = LU$, segue:

 
$M = \begin{bmatrix}
m_{11} & m_{12} & m_{13}\\
m_{21} & m_{22} & m_{23}\\
m_{31} & m_{32} & m_{33}\\
\end{bmatrix} = LU = \begin{bmatrix}
l_{11} & l_{11}*u_{12} & l_{11}*u_{13}\\
l_{21} & l_{21}*u_{12}+l_{22} & l_{21}*u_{13}+l_{22}*u_{23}\\
l_{31} & l_{31}*u_{12} + l_{32} & l_{31}*u_{13} + l_{32}*u_{23} + l_{33}\\
\end{bmatrix} \rightarrow$

### Passo 1 da $L$:  Anda na linha, fixa na coluna
$l_{11} = m_{11}\\
l_{21} = m_{21}\\
l_{31} = m_{31}$

### Passo 1 da U:  Anda na coluna, fixa na linha
$u_{12} = \frac{m_{12}}{l_{11}}\\
u_{13} = \frac{m_{13}}{l_{11}}$

### Passo 2 da L: Anda na linha, fixa na coluna
$l_{22} = m_{22} - l_{21}*u_{12}\\
l_{32} = m_{32} -l_{31}*u_{12}$

### Passo 2 da U: Anda na coluna, fixa na linha
$u_{23} = \frac{m_{23} - l_{21}*u_{13}}{l_{22}}$

### Passo 3 da L: Anda na linha, fixa na coluna
$l_{33} = m_{3} - l_{31}*u_{13} - l_{32}*u_{23}$

### Pseudocódigo desse troço (inicializando L e U com 0):
##### Para 
$k = 1...n:    \\    i = k ... n:\\ % anda na linha
             L[i,k] = M[i,k] - \sum_{r=1}^{k} L[i,r]*U[r,k] \\
             j = k ... n: \\ %anda na coluna
             Se j = k: \\
                     U[j,j] = 1 \\
             Senão: \\  
             U[k,j] = \frac{1}{L_{kk}}(M[k,j] - \sum_{r=1}^{k} *L[k, r]*U[r, j])$\\
             

### Vamos começar com funções separadas para facilitar o entendimento e depois juntamos tudo numa maior.

In [1]:
import numpy as np
import pandas as pd

In [2]:
n = 3 ## tamanho da matriz de exemplo

In [3]:
M = np.zeros((n,n+1)) ##seta uma matriz de exemplo; a coluna neste método tem que ter dimensão n+1 para entrar a resposta
L = np.zeros((n, n+1)) ### matriz inferior com espacinho para a resposta
U = np.zeros((n, n+1)) ### matriz superior com espacinho para a resposta

In [ ]:
M

In [4]:
###preenchendo com os coeficientes e valores da matriz M
M[0][0] = 0.448
M[0][1] = 0.832
M[0][2] = 0.193
M[0][3] = 1
M[1][0] = 0.421
M[1][1] = 0.784
M[1][2] = -0.207
M[1][3] = 2
M[2][0] = -0.319
M[2][1] = 0.884
M[2][2] = 0.279
M[2][3] = 0

##A matriz L também passa por pivotação, então temos que deixar o valor da resposta nela

L[0][3] = 1
L[1][3] = 2
L[2][3] = 0


In [5]:
M_orig = M.copy()

### Pivotação parcial
#### A função abaixo vai encontrar o indice (de linha) do maior valor em módulo na coluna que queremos pivotar, a partir da linha i == j
#### Além disso, faz-se a troca de linhas, de acordo com a coluna j em que estamos

In [ ]:
def pivota(M, n, j):
    max_ = 0
    idx = 0
    for i in range(j,n):
        if abs(M[i,j]) >= max_:
            max_ = M[i,j]
            idx = i
    temp = M[j,:].copy()
    M[j,:] = M[idx,:].copy()
    M[idx,:] = temp.copy()
    return M

####  Vamos pivotar a matriz M inteira para facilitar o funcionamento da decomposição LU. Esta etapa é importante principalmente quando 
#### temos que lidar com erros de arredondamento

In [ ]:
for i in range(n):
    M = pivota(M, n, i)

In [ ]:
[M, L, U]

### Algoritmo de decomposição LU

In [ ]:
for k in range(n):
    for i in range(k, n):
        soma_j = 0
        for r in range(k):
            soma_j = soma_j+L[i, r]*U[r, k]
        L[i,k] = M[i,k] - soma_j

        
    for j in range(k, n): 
        if (j==k):
            U[k,j] = 1
        else:
            soma_i = 0
            for r in range(k):
                soma_i = soma_i + L[k, r]*U[r, j]
            U[k, j] = (1/L[k,k])*(M[k,j]-soma_i)


In [ ]:
[M, L, U]

In [ ]:
np.dot(L[:,0:3],U[:,0:3])

In [ ]:
M  ###Funcionou! 

### Precisamos agora resolver o problema LC =  B. Vamos olhar a matriz L:

In [ ]:
pd.DataFrame(L)

### COMO M FOI PIVOTADA, É CRUCIAL QUE L RECEBA AS RESPOSTAS NA MESMA ORDEM DA PIVOTAÇÃO EM M

In [ ]:
L[:,n] = M[:,n] 

In [ ]:
L

### O algoritmo de solução deste sistema é o de substituições sucessivas (exatamente o inverso da retrossubstituição sucessiva).


$\begin{equation}
\left[
    \begin{array}{ccc:c}
        L[0,0]x_1 & 0 & 0 & L[0,3]\\
        L[1,0]x_1 & L[1,1]x_2 & 0 & L[1,3]\\
        L[2,0]x_1 & L[2,1]x_2 & L[2,2]x_3 & L[2,3]\\
    \end{array}
\right]
\end{equation}$

### Neste caso, teríamos:

$\begin{equation}
x_1 = \frac{L[0,3]}{L[0,0]}\\
x_2  = \frac{L[1,3]}{L[1,1]} - \frac{L[1,0]*x_1}{L[1,1]}\\
x_3  = \frac{L[2,3]}{L[2,2]} - \frac{L[2,0]*x_1}{L[2,2]} - \frac{L[2,1]*x_2}{L[2,2]}
\end{equation}$

### Note que os coeficientes que acompanham as incógnitas $x_i$ andam até o limite (i) nas colunas; ex: na linha 1, 0 passos; na linha 2, 1 passo (até L[1,0]); na linha 3, 2 passos (até L[2,1]). 

In [ ]:
c = np.zeros((n,1)) ##vetor que vai receber a resposta

In [ ]:
### começamos com i = 0; andamos de cima pra baixo na matriz
c[0] = L[0,n]/L[0,0]

In [ ]:
for i in range(1, n): ###restringimos os casos a partir de 1, pois i = 0 ja foi calculado
    termo_ind = L[i,n]/L[i,i] ###termo independente da equacao
    soma = 0
    for j in range(i):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
        soma += ((L[i,j])/(L[i,i]))*c[j]
    c[i] = termo_ind - soma          ####o vetor de respostas vai sendo alimentado do começo ao fim

In [ ]:
c

### Agora vamos resolver o problema UX = C

### Como U é triangular superior, conseguimos aproveitar o algoritmo de retrossubstituições sucessivas:

In [ ]:
x = np.zeros((n, 1)) ##vetor que vai receber a solução do sistema 

In [ ]:
U[:,n] = c.reshape(1,-1) ###atribuindo o valor de C na última coluna de U

In [ ]:
## começamos com o caso de i = n (andamos de baixo pra cima na matriz)
x[n-1] = U[n-1, n]/U[n-1, n-1]

In [ ]:
for i in range(1, n): ###restringimos os casos a partir de n-1. o indexador do python ja fa isso naturalmente [1,n) ==[1,n-1]
    termo_ind = U[n-i-1,n]/U[n-i-1,n-i-1] 
    soma = 0
    for j in range(n-i, n):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
        soma += ((U[n-i-1,j])/(U[n-i-1,n-i-1]))*x[j]
    x[n-i-1] = termo_ind - soma          ####o vetor de respostas vai sendo alimentado de tras pra frente tambem

In [ ]:
x

In [ ]:
pd.DataFrame(np.dot(M_orig[:,0:n],x).round(3)) #Sim! Deu certo. Guardamos essa M não pivoatada para checar

# Agrupando tudo em funções

In [6]:
def pivotacao(M, n):
    for j in range(n):
        max_ = 0
        idx = 0
        for i in range(j,n):
            if abs(M[i,j]) >= max_:
                max_ = M[i,j]
                idx = i
        temp = M[j,:].copy()
        M[j,:] = M[idx,:].copy()
        M[idx,:] = temp.copy()
    return M

In [7]:
M = pivotacao(M, n)

In [8]:
def decomposicao_LU(M, n):
    L = np.zeros((M.shape))
    U = np.zeros((M.shape))


    for k in range(n):
        for i in range(k, n):
            soma_j = 0
            for r in range(k):
                soma_j = soma_j+L[i, r]*U[r, k]
            L[i,k] = M[i,k] - soma_j

        
        for j in range(k, n): 
            if (j==k):
                U[k,j] = 1
            else:
                soma_i = 0
                for r in range(k):
                    soma_i = soma_i + L[k, r]*U[r, j]
                U[k, j] = (1/L[k,k])*(M[k,j]-soma_i)
    L[:,n] = M[:,n]
    return [M, L, U]

In [9]:
sols = decomposicao_LU(M,n)
sols

[array([[ 0.448,  0.832,  0.193,  1.   ],
        [-0.319,  0.884,  0.279,  0.   ],
        [ 0.421,  0.784, -0.207,  2.   ]]),
 array([[ 0.448     ,  0.        ,  0.        ,  1.        ],
        [-0.319     ,  1.47642857,  0.        ,  0.        ],
        [ 0.421     ,  0.00214286, -0.3889727 ,  2.        ]]),
 array([[1.        , 1.85714286, 0.43080357, 0.        ],
        [0.        , 1.        , 0.28204977, 0.        ],
        [0.        , 0.        , 1.        , 0.        ]])]

In [10]:
def solucao(L, n):
    c = np.zeros((n, 1))
    c[0] = L[0,n]/L[0,0]
    for i in range(1, n): ###restringimos os casos a partir de 1, pois i = 0 ja foi calculado
        termo_ind = L[i,n]/L[i,i] ###termo independente da equacao
        soma = 0
        for j in range(i):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
            soma += ((L[i,j])/(L[i,i]))*c[j]
        c[i] = termo_ind - soma        ####o vetor de respostas vai sendo alimentado do começo ao fim
    return c
    

In [11]:
c = solucao(sols[1], n)

In [12]:
def retro_solucao(U, n, c):
    x = np.zeros((n, 1))
    U[:,n] = c.reshape(1, -1)
    
    x[n-1] = U[n-1, n]/U[n-1, n-1] ## começamos com o caso de i = n (andamos de baixo pra cima na matriz)
    
    for i in range(1, n): ###restringimos os casos a partir de n-1. o indexador do python ja fa isso naturalmente [1,n) ==[1,n-1]
        termo_ind = U[n-i-1,n]/U[n-i-1,n-i-1] 
        soma = 0
        for j in range(n-i, n):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
            soma += ((U[n-i-1,j])/(U[n-i-1,n-i-1]))*x[j]
        x[n-i-1] = termo_ind - soma          ####o vetor de respostas vai sendo alimentado de tras pra frente tambem
    return x

In [14]:
X = retro_solucao(sols[2], n, c)

### Testando se funcionou:

In [19]:
np.dot(M_orig[:,0:n], X).round(3)  ##sim deu certo!

array([[1.],
       [2.],
       [0.]])

In [ ]:
### No link abaixo tem a implementação do método de Doolittle, que é um pouco diferente mas faz algo parecido
### https://www.geeksforgeeks.org/doolittle-algorithm-lu-decomposition/